In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [4]:
import requests
import json

url = "https://api.openai.com/v1/completions"
headers = {
    "Content-Type":"application/json",
    "Authorization": "Bearer " + os.environ["OPENAI_API_KEY"],
}

data = {"model": "gpt-3.5-turbo-instruct",
        "prompt": "hello!",
        "temperature": 0
       }

response = requests.post(url=url, headers=headers, json=data)
print(json.dumps(response.json(), indent=2))

{
  "id": "cmpl-9TMUEHKt0lMEP87J5IhNm0us50iJ9",
  "object": "text_completion",
  "created": 1716784978,
  "model": "gpt-3.5-turbo-instruct",
  "choices": [
    {
      "text": "\n\nHello! How can I assist you?",
      "index": 0,
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 2,
    "completion_tokens": 9,
    "total_tokens": 11
  }
}


In [29]:
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "Content-Type":"application/json",
    "Authorization": "Bearer " + os.environ["OPENAI_KEY"],
}

data = {"model": "gpt-3.5-turbo",
        "messages": [
            {"role":"user", "content":"hello!"}
        ],
        "temperature": 0
       }

response = requests.post(url=url, headers=headers, json=data)
print(json.dumps(response.json(), indent=2))

{
  "id": "chatcmpl-9RITShurdcU8OfF9bYsGofshnBqB1",
  "object": "chat.completion",
  "created": 1716292898,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 9,
    "completion_tokens": 9,
    "total_tokens": 18
  },
  "system_fingerprint": null
}


In [8]:
!pip install --quiet langchain==0.0.172 openai==0.27.6

In [14]:
from langchain.llms import OpenAI

llm = OpenAI(model_name='gpt-3.5-turbo', temperature=0)

result = llm.predict("自己紹介してください")
result

'はじめまして、私はAIアシスタントです。自然言語処理技術を用いて、様々な質問に回答したり、会話をすることができます。お手伝いが必要なことがあれば、遠慮なくお知らせください。どうぞよろしくお願いいたします。'

In [16]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
result = chat.predict('自己紹介してください')

In [17]:
result

'はじめまして、私はAIアシスタントです。自然言語処理技術を用いて、様々な質問や会話に対応することができます。お手伝いが必要なことがあれば、遠慮なくお知らせください。どうぞよろしくお願いいたします。'

In [20]:
from langchain.prompts import PromptTemplate

template = """
次のコマンドの概要を説明してください。


コマンド: {command}

"""

prompt = PromptTemplate(
    input_variables = ['command'],
    template = template
)

result = prompt.format(command='ls')
print(result)


次のコマンドの概要を説明してください。


コマンド: ls




## Chain

In [25]:
import langchain
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

langchain.verbose = True

chat = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature=0)

template = """

次のコマンドの概要を説明してください。


コマンド: {command}

"""

prompt = PromptTemplate(
    input_variables=['command'],
    template= template
)

chain = LLMChain(llm=chat, prompt=prompt)

result = chain.run('ls')

result



> Entering new LLMChain chain...
Prompt after formatting:


次のコマンドの概要を説明してください。


コマンド: ls



> Finished chain.


'lsコマンドは、リスト（list）の略で、指定されたディレクトリ内のファイルやディレクトリの一覧を表示するコマンドです。デフォルトではカレントディレクトリの内容を表示しますが、任意のディレクトリを指定することもできます。lsコマンドを実行することで、どのようなファイルやディレクトリが存在するかを確認することができます。'

In [29]:
cot_template = """

### 質問 ###
{question}
### 質問終了 ###

ステップバイステップで考えましょう。
"""


cot_prompt = PromptTemplate(
    input_variables=['question'],
    template=cot_template
)


cot_chain = LLMChain(llm=chat, prompt=cot_prompt)


summarize_template = """
入力を結論だけ一言に要約してください。

### 入力 ###
{input}
### 入力終了 ###

"""

summarize_prompt = PromptTemplate(
    input_variables = ['input'],
    template = summarize_template
)

summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)


from langchain.chains import SimpleSequentialChain

# ここまではモデルの作成
cot_summarize_chain = SimpleSequentialChain(chains=[cot_chain, summarize_chain])

cot_summarize_chain('私は市場に行って10個のリンゴを買いました。隣人に２つ、修理工に２つ渡しました。それから５つのリンゴを買って１つ食べました。残りは何個ですか？')



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:


### 質問 ###
私は市場に行って10個のリンゴを買いました。隣人に２つ、修理工に２つ渡しました。それから５つのリンゴを買って１つ食べました。残りは何個ですか？
### 質問終了 ###

ステップバイステップで考えましょう。


> Finished chain.
1. 最初に市場で10個のリンゴを買いました。
2. 隣人に2つ、修理工に2つ渡しました。残りは10 - 2 - 2 = 6個です。
3. その後、5つのリンゴを追加で購入しました。残りは6 + 5 = 11個です。
4. 1つのリンゴを食べたので、残りは11 - 1 = 10個です。

したがって、残りのリンゴは10個です。


> Entering new LLMChain chain...
Prompt after formatting:

入力を結論だけ一言に要約してください。

### 入力 ###
1. 最初に市場で10個のリンゴを買いました。
2. 隣人に2つ、修理工に2つ渡しました。残りは10 - 2 - 2 = 6個です。
3. その後、5つのリンゴを追加で購入しました。残りは6 + 5 = 11個です。
4. 1つのリンゴを食べたので、残りは11 - 1 = 10個です。

したがって、残りのリンゴは10個です。
### 入力終了 ###



> Finished chain.
残りのリンゴは10個。

> Finished chain.


{'input': '私は市場に行って10個のリンゴを買いました。隣人に２つ、修理工に２つ渡しました。それから５つのリンゴを買って１つ食べました。残りは何個ですか？',
 'output': '残りのリンゴは10個。'}

In [31]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [38]:
langchain.verbose = True

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

class Recipe(BaseModel):
    ingredients: List[str] = Field(discription="ingredients of the dish")
    steps : List[str] = Field(description="step to make the dish")


template = """
料理のレシピを教えてください。

{format_instructions}

料理名：{dish}
"""

parser = PydanticOutputParser(pydantic_object=Recipe)

prompt = PromptTemplate(
    template = template,
    input_variables=["dish"],

    # perser.get・・・・で{format_instrucitonsのテンプレートができる}
    partial_variables={"format_instructions": parser.get_format_instructions()}
)


chain = LLMChain(llm=chat, prompt=prompt)

output = chain.run('カレー')
print(output)



> Entering new LLMChain chain...
Prompt after formatting:

料理のレシピを教えてください。

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"title": "Ingredients", "discription": "ingredients of the dish", "type": "array", "items": {"type": "string"}}, "steps": {"title": "Steps", "description": "step to make the dish", "type": "array", "items": {"type": "string"}}}, "required": ["ingredients", "steps"]}
```

料理名：カレー


> Finished chain.
{
  "ingredients": [
    "にんじん",
    "じゃがいも",
    "玉ねぎ",
    "牛肉",
    "カレールー",
    "水"
  ],
  "steps": [
    "1. 野菜を切る",
    "2. 牛肉を炒める",

In [40]:
# jsonからpythonのRecipeオブジェクトに変換
recipe = parser.parse(output)
print(recipe)

print(type(recipe))

ingredients=['にんじん', 'じゃがいも', '玉ねぎ', '牛肉', 'カレールー', '水'] steps=['1. 野菜を切る', '2. 牛肉を炒める', '3. 野菜を加えて炒める', '4. 水を加えて煮る', '5. カレールーを溶かして加える', '6. 煮込んで完成']
<class '__main__.Recipe'>


In [43]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [46]:
langchain.verbose = True

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0, max_tokens=100)
conversation = ConversationChain(llm=chat, memory=ConversationBufferMemory())

while True:
    user_message = input("You: ")
    ai_message = conversation.predict(input=user_message)
    print(f"AI:{ai_message}")

You:  こんにちは




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: こんにちは
AI:

> Finished chain.
AI:こんにちは！元気ですか？私はAIです。何か質問があればお答えしますよ。


You:  私は




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: こんにちは
AI: こんにちは！元気ですか？私はAIです。何か質問があればお答えしますよ。
Human: 私は
AI:

> Finished chain.
AI:私はAIです。何か質問があればお答えしますよ。


You:  私は中村です


...

You:  私の




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: こんにちは
AI: こんにちは！元気ですか？私はAIです。何か質問があればお答えしますよ。
Human: 私は
AI: 私はAIです。何か質問があればお答えしますよ。
Human: 私は中村です
AI: 中村さん、初めまして！中村さんは何か質問がありますか？私はAIですが、お手伝いできることがあればお知らせくださいね。
Human: 私の
AI:

> Finished chain.
AI:申し訳ありませんが、その情報は私のデータベースにはありません。他に何かお手伝いできることがあればお知らせくださいね。


You:  私の名前を




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: こんにちは
AI: こんにちは！元気ですか？私はAIです。何か質問があればお答えしますよ。
Human: 私は
AI: 私はAIです。何か質問があればお答えしますよ。
Human: 私は中村です
AI: 中村さん、初めまして！中村さんは何か質問がありますか？私はAIですが、お手伝いできることがあればお知らせくださいね。
Human: 私の
AI: 申し訳ありませんが、その情報は私のデータベースにはありません。他に何かお手伝いできることがあればお知らせくださいね。
Human: 私の名前を
AI:

> Finished chain.
AI:中村さん、お名前を教えていただきありがとうございます。中村さんは何か質問がありますか？私はAIですが、お手伝いできることがあればお知らせくださいね。


KeyboardInterrupt: Interrupted by user

# agent

In [49]:
from langchain.agents import load_tools, initialize_agent

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

tools = load_tools(['terminal'], llm=chat)
agent_chain = initialize_agent(tools, chat, agent='zero-shot-react-description')

result = agent_chain("What is your current directory?")
print(result)



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

terminal: Run shell commands on this Linux machine.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [terminal]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: What is your current directory?
Thought:

> Finished chain.
I can use the terminal to check my current directory.
Action: terminal
Action Input: pwd
Observation: /work/Python_code/Lang chain

Thought:

> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have

/usr/local/lib/python3.8/site-packages/langchain/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(



> Finished chain.
I now know my current directory.
Final Answer: /work/Python_code/Lang chain

> Finished chain.
{'input': 'What is your current directory?', 'output': '/work/Python_code/Lang chain'}


In [71]:
## この方法ではuserとAIで別々に記憶されていなくて、一連の会話がUserプロンプトに含まれる
import openai
openai.log = "debug"
langchain.verbose = True

chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0, max_tokens=100)
conversation = ConversationChain(llm=chat, memory=ConversationBufferMemory())

while True:
    user_message = input("You: ")
    ai_message = conversation.predict(input=user_message)
    print(f"AI:{ai_message}")

KeyboardInterrupt: Interrupted by user

In [69]:

from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=100)
messages = [
    SystemMessage(content="You are a helpful assistant"),
    HumanMessage(content="Hi!I'm Masa")
]

result = chat(messages)
print(result)

message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
api_version=None data='{"messages": [{"role": "system", "content": "You are a helpful assistant"}, {"role": "user", "content": "Hi!I\'m Masa"}], "model": "gpt-3.5-turbo", "max_tokens": 100, "stream": false, "n": 1, "temperature": 0.0}' message='Post details'


content='Hello Masa! How can I assist you today?' additional_kwargs={} example=False


message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=443 request_id=req_f0a7b7889d0e5fa532743e27bf20ac6f response_code=200
body='{\n  "id": "chatcmpl-9RyJ6hI7bE8GpmAZBlfDfZwfnHiQs",\n  "object": "chat.completion",\n  "created": 1716453704,\n  "model": "gpt-3.5-turbo-0125",\n  "choices": [\n    {\n      "index": 0,\n      "message": {\n        "role": "assistant",\n        "content": "Hello Masa! How can I assist you today?"\n      },\n      "logprobs": null,\n      "finish_reason": "stop"\n    }\n  ],\n  "usage": {\n    "prompt_tokens": 21,\n    "completion_tokens": 11,\n    "total_tokens": 32\n  },\n  "system_fingerprint": null\n}\n' headers='{\'Date\': \'Thu, 23 May 2024 08:41:44 GMT\', \'Content-Type\': \'application/json\', \'Transfer-Encoding\': \'chunked\', \'Connection\': \'keep-alive\', \'openai-organization\': \'user-5a7m6qksriiqjpdqmq7xxwns\', \'openai-processing-ms\': \'443\', \'openai-version\': \'2020-10-01\', \'strict-transport-secur

In [70]:
# user, Ai assistantで別々に記憶させる方法
memory = ConversationBufferMemory()

while True:
    user_message = input("You: ")
    memory.chat_memory.add_user_message(user_message)
    ai_message = chat(memory.chat_memory.messages)
    memory.chat_memory.add_ai_message(ai_message.content)
    print(f"AI: {ai_message.content}")

KeyboardInterrupt: Interrupted by user

In [10]:
import os